# Automated Machine Learning (AutoML) Search

## Background

### Machine Learning

[Machine learning](https://en.wikipedia.org/wiki/Machine_learning) (ML) is the process of constructing a mathematical model of a system based on a sample dataset collected from that system.

A main goal of training an ML model is to teach the model to separate the signal present in the data from the noise inherent in system and in the data collection process. If this is done effectively, the model can then be used to make accurate predictions about the system when presented with new, similar data. Additionally, introspecting on an ML model can reveal key information about the system being modeled, such as which inputs and transformations of the inputs are most useful to the ML model for learning the signal in the data, and are therefore the most predictive.

There are [a variety](https://en.wikipedia.org/wiki/Machine_learning#Approaches) of ML problem types. Supervised learning describes the case where the collected data contains an output quantity to be modeled and a set of inputs with which to train the model. EvalML focuses on training models for the supervised learning case.

EvalML supports three common supervised ML problem types. The first is regression, where the target quantity to model is a continuous numeric value. Next are binary and multiclass classification, where the target quantity to model consists of two or more discrete values or categories. The choice of which supervised ML problem type is most appropriate depends on domain expertise and on how the model will be evaluated and used.


### AutoML and Search

[AutoML](https://en.wikipedia.org/wiki/Automated_machine_learning) is the process of automating the construction, training and evaluation of ML models. Given a data and some configuration, AutoML searches for the most effective and accurate ML model or models to fit the dataset. During the search, AutoML will explore different combinations of model type, model parameters and model architecture.

An effective AutoML solution offers several advantages over constructing and tuning ML models by hand. AutoML can assist with many of the difficult aspects of ML, such as avoiding overfitting and underfitting, imbalanced data, detecting data leakage and other potential issues with the problem setup, and automatically applying best-practice data cleaning, feature engineering, feature selection and various modeling techniques. AutoML can also leverage search algorithms to optimally sweep the hyperparameter search space, resulting in model performance which would be difficult to achieve by manual training.

## AutoML in EvalML

EvalML supports all of the above and more.

In its simplest usage, the AutoML search interface requires only the input data, the target data and a `problem_type` specifying what kind of supervised ML problem to model.

In [1]:
import evalml

X, y = evalml.demos.load_breast_cancer()

automl = evalml.automl.AutoMLSearch(problem_type='binary')
automl.search(X, y)

Using default limit of max_pipelines=5.

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Searching up to 5 pipelines. 
Allowed model families: xgboost, catboost, random_forest, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

(1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.660
(2/5) CatBoost Classifier w/ Simple Imputer    Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.094
(3/5) XGBoost Classifier w/ Simple Imputer     Elapsed:00:05
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.101
(4/5) Random Forest Classifier w/ Simple Im... Elapsed:00:05
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.123
(5/5) Logistic Regression Classifier w/ Sim... Elapsed:00:06
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.091

Search finished after 00:07            
Best pipeline: Logistic Regression Classifier w/ Simple Imputer + Standard Scaler
Best pipeline Log Loss Binary: 0.091164


The AutoML search will log its progress, reporting each pipeline and parameter set evaluated during the search.

By default, AutoML will search a fixed number of pipeline and parameter pairs (5). The first pipeline to be evaluated will always be a baseline model representing a trivial solution.

The AutoML interface supports a variety of other parameters. For a comprehensive list, please [refer to the API docs.](../generated/evalml.pipelines.automl.AutoMLSearch.ipynb)

## View Rankings
A summary of all the pipelines built can be returned as a pandas DataFrame. It is sorted by score. EvalML knows based on our objective function whether higher or lower is better.

In [3]:
automl.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,4,Logistic Regression Classifier w/ Simple Imput...,0.091164,False,{'Simple Imputer': {'impute_strategy': 'most_f...
1,1,CatBoost Classifier w/ Simple Imputer,0.093553,False,{'Simple Imputer': {'impute_strategy': 'most_f...
2,2,XGBoost Classifier w/ Simple Imputer,0.100965,True,{'Simple Imputer': {'impute_strategy': 'most_f...
3,3,Random Forest Classifier w/ Simple Imputer,0.122537,False,{'Simple Imputer': {'impute_strategy': 'most_f...
4,0,Mode Baseline Binary Classification Pipeline,0.660321,False,{'Baseline Classifier': {'strategy': 'random_w...


## Describe Pipeline
Each pipeline is given an `id`. We can get more information about any particular pipeline using that `id`. Here, we will get more information about the pipeline with `id = 0`.

In [4]:
automl.describe_pipeline(1)

*****************************************
* CatBoost Classifier w/ Simple Imputer *
*****************************************

Problem Type: Binary Classification
Model Family: CatBoost

Pipeline Steps
1. Simple Imputer
	 * impute_strategy : most_frequent
	 * fill_value : None
2. CatBoost Classifier
	 * n_estimators : 1000
	 * eta : 0.03
	 * max_depth : 6
	 * bootstrap_type : None

Training
Training for Binary Classification problems.
Total training time (including CV): 5.3 seconds

Cross Validation
----------------
             Log Loss Binary  Accuracy Binary  Balanced Accuracy Binary    F1  Precision   AUC  MCC Binary # Training # Testing
0                      0.106            0.958                     0.949 0.967      0.951 0.995       0.910    379.000   190.000
1                      0.082            0.979                     0.975 0.983      0.975 0.994       0.955    379.000   190.000
2                      0.093            0.974                     0.976 0.979      0.991 0.990

## Get Pipeline
We can get the object of any pipeline via their `id` as well:

In [8]:
pipeline = automl.get_pipeline(1)
print(pipeline.name)
print(pipeline.parameters)

CatBoost Classifier w/ Simple Imputer
{'Simple Imputer': {'impute_strategy': 'most_frequent', 'fill_value': None}, 'CatBoost Classifier': {'n_estimators': 1000, 'eta': 0.03, 'max_depth': 6, 'bootstrap_type': None}}


### Get best pipeline
If we specifically want to get the best pipeline, there is a convenient accessor for that.

In [9]:
best_pipeline = automl.best_pipeline
print(best_pipeline.name)
print(best_pipeline.parameters)

Logistic Regression Classifier w/ Simple Imputer + Standard Scaler
{'Simple Imputer': {'impute_strategy': 'most_frequent', 'fill_value': None}, 'Logistic Regression Classifier': {'penalty': 'l2', 'C': 1.0, 'n_jobs': -1}}


## Access raw results

The `AutoMLSearch` class records detailed results information under the `results` field, including information about the cross-validation scoring and parameters.

In [11]:
automl.results

{'pipeline_results': {0: {'id': 0,
   'pipeline_name': 'Mode Baseline Binary Classification Pipeline',
   'pipeline_class': evalml.pipelines.classification.baseline_binary.ModeBaselineBinaryPipeline,
   'pipeline_summary': 'Baseline Classifier',
   'parameters': {'Baseline Classifier': {'strategy': 'random_weighted'}},
   'score': 0.660320827581381,
   'high_variance_cv': False,
   'training_time': 0.02433300018310547,
   'cv_data': [{'all_objective_scores': OrderedDict([('Log Loss Binary',
                   0.6608932451679239),
                  ('Accuracy Binary', 0.6263157894736842),
                  ('Balanced Accuracy Binary', 0.5),
                  ('F1', 0.7702265372168284),
                  ('Precision', 0.6263157894736842),
                  ('AUC', 0.5),
                  ('MCC Binary', 0.0),
                  ('# Training', 379),
                  ('# Testing', 190)]),
     'score': 0.6608932451679239,
     'binary_classification_threshold': 0.5},
    {'all_objective_sco